# Character Level CNN Variational Autoencoder

In [1]:
import Corpus.gutenberg as corpus
from TextPreprocess.Tokenizer.Stanford import tokenize
from Utils.visual import hist, tally
from Utils.debug import dump
from Utils.generator import sliding_window, random_window, transform
from Utils.FS.file import save, load
from Utils.keras import compact_embedding
from Utils.misc import batch
from Utils.indexer import build_index
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
data = tokenize(corpus.raw())

In [3]:
from collections import Counter

def generator():
    for word in data:
        for char in word:
            yield char

s2i, i2s, size = build_index(generator())

In [4]:
MAX_SEQ_LENGTH = max([len(word) for word in data])
MAX_SEQ_LENGTH = 10

48

In [5]:
NUM_SYMBOL = size
NUM_SYMBOL

87

In [17]:
X_train_unpad = [batch(s2i, w) for w in generator()]
X_train = pad_sequences(X_train_unpad, maxlen=MAX_SEQ_LENGTH, dtype='int32',
    padding='pre', truncating='pre', value=0)

X_train

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  2],
       ..., 
       [ 0,  0,  0, ...,  0,  0, 34],
       [ 0,  0,  0, ...,  0,  0,  8],
       [ 0,  0,  0, ...,  0,  0, 45]], dtype=int32)

In [ ]:
from keras.layers import Input, Convolution1D, MaxPooling1D, Dense, Flatten, Dropout, Embedding,BatchNormalization
from keras.models import Model, Sequential
from keras.regularizers import l2

def create_baseline(dropout=0, branching=5):
    
    sequence_input = Input(shape=(MAX_SEQ_LENGTH, NUM_SYMBOL), dtype='int32')
    
    #print(embedded_sequences)
    x = Convolution1D(300, 3, activation='selu')(sequence_input)
    x = Convolution1D(10, 3, activation='selu')(x)
    #x = Convolution1D(300, branching)(x)
    #x = Convolution1D(300, branching)(x)
    #x = Convolution1D(300, branching)(x)
    #x = Convolution1D(300, branching)(x)
    #x = Convolution1D(300, branching)(x)
    #x = Convolution1D(300, branching)(x)
    
    #x = MaxPooling1D()(x)
    #x = Dropout(dropout)(x)
    
    """
    x = Convolution1D(800, 2, activation='relu')(x)
    x = MaxPooling1D()(x)
    x = Dropout(dropout)(x)
    """
    
    x = Flatten()(embedded_sequences)
    
    """
    x = Dense(3000, activation='tanh')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    
    x = Dense(1500, activation='tanh')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
    """
    
    x = Dense(10, activation='selu')(x)
    #x = BatchNormalization()(x)
    #x = Dropout(dropout)(x)
    
    #x = Dense(10, activation='selu')(x)
    #x = BatchNormalization()(x)
    #x = Dropout(dropout)(x)
    
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])
    return model

In [ ]:
from keras_tqdm import TQDMNotebookCallback

BATCH_SIZE = 1024
DROPOUT = 0.5
BRANCHING = 2

model = create_baseline(DROPOUT, BRANCHING)
model.fit(
    x = X_train,
    y = y,
    batch_size=1, 
    epochs=200,
    #validation_split=0.2,
    shuffle=True
    #class_weight=class_weight
    #verbose=0, callbacks=[TQDMNotebookCallback()]
)